In [1]:
import pandas as pd
import cv2 as cv

In [2]:
train_df = pd.read_csv("./dataset/splits/train.csv")
train_df.head()

,Unnamed: 0,clip_name,clip_path,label,encoded_label
0,0,v_Diving_g07_c04.avi,./dataset/Diving/v_Diving_g07_c04.avi,Diving,0
1,1,v_Diving_g05_c02.avi,./dataset/Diving/v_Diving_g05_c02.avi,Diving,0
2,2,v_Diving_g22_c05.avi,./dataset/Diving/v_Diving_g22_c05.avi,Diving,0
3,3,v_Diving_g12_c02.avi,./dataset/Diving/v_Diving_g12_c02.avi,Diving,0
4,4,v_Diving_g07_c01.avi,./dataset/Diving/v_Diving_g07_c01.avi,Diving,0


In [5]:
def manual_normalize(frame):
    mean, std = cv.meanStdDev(frame)
    mean = mean.flatten()
    std = std.flatten()
    return (resized - mean) / std

def open_cv_normalize(frame):
    return cv.normalize(frame, None, alpha=0, beta=1.0, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

def open_cv_normalization_255(frame):
    return cv.normalize(frame, None, alpha=0, beta=255,
                               norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)


In [6]:
file = cv.VideoCapture(train_df.iloc[0].clip_path)

if not file:
    print("Error opening video stream or file")

while file.isOpened():
    ret, frame = file.read()

    if ret:
        resized = cv.resize(frame, (224, 224), interpolation=cv.INTER_CUBIC)

        resized = manual_normalize(resized)

        print(resized.shape)
        cv.imshow('Frame', frame)
        cv.imshow('Resized', resized)
        # cv.imshow('Resized2', resized2)
        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

file.release()
cv.destroyWindow('Frame')
cv.destroyWindow('Resized')
# cv.destroyWindow('Resized2')


(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)


QFontDatabase: Cannot find font directory /home/yogendra/workspace/python/university/Video-Analytics-Assignment1/.venv/lib/python3.12/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/yogendra/workspace/python/university/Video-Analytics-Assignment1/.venv/lib/python3.12/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/yogendra/workspace/python/university/Video-Analytics-Assignment1/.venv/lib/python3.12/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/yogendra/workspace/python/university/Video-Analytics-Assignment1/.venv/lib/python3.12/site-pac

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 